## 决策树-代码实现


### 思路

#### 一、构建决策树

##### 决策树的数据结构：


![image.png](./2.png)

使用嵌套的字典来记录树

![image.png](./4.png)

对于上图这样一个决策树，其用字典表示为：

```
{
    "特征1":{
                "取值1":{子树1},
                "取值2":{子树2}
            }
}
```

##### 递归构建决策树


```
createTree(dataset)····················································【dataset的每行是一组数据，每列为一种特征; 返回值是一个字典myTree】
    
    1、遍历所有特征,找出最优特征 bestFeature
       myTree={bestFeature:{}} ········································【使用bestFeature作为根节点，即字典的key】
    

    2、找出bestFeature的取值列表 bestFeatureValues
    
    3、for value in bestFeatureValues:

          从dataset中，选出 bestFeature=value 的数据，得到newDataset·····【根据bestFeature的取值，划分原数据集为子集newDataSet】
        
          myTree[bestFeature][value]=createTree(newDataset)·············【递归，从newDataSet构建子树】

    4、return myTree
```


##### 递归的停止条件

1. 遍历dataset的标签列：

    如果dataset中的所有数据都属于一个标签，说明分类完成，是一个叶子结点
    
        return 当前dataset所属的标签类别

2. 如果所有特征都遍历了，此时仍然有多种特征

    也就是说，存在至少两组数据，他们的所有特征都一样，但是他们所属的标签类别不一样

        return 当前dataset中更多的标签类别

##### 找最优特征（手册6.1.3【chooseBestFeatureToSplit(dataSet)】）

遍历所有特征，计算信息增益

1. 计算数据集的信息熵····································计算信息熵：手册6.1.2节【calcShannonEnt(dataSet)】函数
2. 对于每一个特征：
    1. 根据特征的不同取值划分子集························构建子集：手册上6.1.3节【splitDataSet(dataSet, axis, value)】函数
    2. 计算子集的平均信息熵
    3. 计算信息增益=子集的平均信息熵-原数据集的信息熵
    4. 信息增益最大的为最优特征

#### 二、根据决策树对数据分类（手册6.1.5【classify(inputTree,labels, testVec)】）

递归地访问决策树字典直到访问到叶子结点，

判断叶子结点：叶子结点是标签类别而不是字典



### 代码演示

#### 1、导入数据

![image.png](5.png)

In [5]:
# 函数说明:创建测试数据集
def createDataSet():
    """
    Parameters:
    无
    Returns:
    dataSet - 数据集
    labels - 分类属性
    """
    dataSet = [[1, 1, 'yes'],#数据集
            [1, 1, 'yes'],
            [1, 0, 'no'],
            [0, 1, 'no'],
            [0, 1, 'no']]
    labels = ['no surfacing', 'flippers']#分类属性
    return dataSet, labels#返回数据集和分类属性

dataSet, labels = createDataSet()
print(labels)
print(dataSet)

['no surfacing', 'flippers']
[[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]


#### 2、构建决策树

##### 选择最优特征

In [6]:
from math import log
# 函数说明:计算给定数据集的经验熵(香农熵)
def calcShannonEnt(dataSet):
    """
    Parameters:
    dataSet - 数据集
    Returns:
    shannonEnt - 经验熵(香农熵)
    """
    numEntires = len(dataSet) #返回数据集的行数
    labelCounts = {} #保存每个标签(Label)出现次数的字典
    for featVec in dataSet: #对每组特征向量进行统计
        currentLabel = featVec[-1] #提取标签(Label)信息
        if currentLabel not in labelCounts.keys(): #如果标签(Label)没有放入统计次数的字典,添加进去
            labelCounts[currentLabel] = 0
        labelCounts[currentLabel] += 1 #Label计数
    shannonEnt = 0.0 #经验熵(香农熵)
    for key in labelCounts: #计算香农熵
        prob = float(labelCounts[key]) / numEntires #选择该标签(Label)的概率
        shannonEnt -= prob * log(prob, 2) #利用公式计算
    return shannonEnt #返回经验熵(香农熵)


# 函数说明:按照给定特征划分数据集
def splitDataSet(dataSet, axis, value):
    """
    Parameters:
    dataSet - 待划分的数据集
    axis - 划分数据集的特征
    value - 需要返回的特征的值
    Returns:
    无
    64
    6.1 决策树
    """
    retDataSet = [] #创建返回的数据集列表
    for featVec in dataSet: #遍历数据集
        if featVec[axis] == value:
            reducedFeatVec = featVec[:axis] #去掉axis特征
            reducedFeatVec.extend(featVec[axis+1:])#将符合条件的添加到返回的数据集
            retDataSet.append(reducedFeatVec)
    return retDataSet

def chooseBestFeatureToSplit(dataSet):
    """
    Parameters:
    dataSet - 数据集
    Returns:
    bestFeature - 信息增益最大的(最优)特征的索引值
    """
    # 函数说明:选择最优特征
    numFeatures = len(dataSet[0]) - 1 #特征数量
    baseEntropy = calcShannonEnt(dataSet) #计算数据集的香农熵
    bestInfoGain = 0.0 #信息增益
    bestFeature = -1 #最优特征的索引值
    for i in range(numFeatures): #遍历所有特征
        #获取dataSet的第i个所有特征
        featList = [example[i] for example in dataSet]
        uniqueVals = set(featList) #创建set集合{},元素不可重复
        newEntropy = 0.0 #经验条件熵
        for value in uniqueVals: #计算信息增益
            subDataSet = splitDataSet(dataSet, i, value) #subDataSet划分后的子集
            prob = len(subDataSet) / float(len(dataSet)) #计算子集的概率
            newEntropy += prob * calcShannonEnt(subDataSet)#根据公式计算经验条件熵
        infoGain = baseEntropy - newEntropy #信息增益
        #print("第%d个特征的增益为%.3f" % (i, infoGain)) #打印每个特征的信息增益
        if (infoGain > bestInfoGain): #计算信息增益
            bestInfoGain = infoGain #更新信息增益，找到最大的信息增益
            bestFeature = i #记录信息增益最大的特征的索引值
    return bestFeature #返回信息增益最大的特征的索引值


print(f"当前数据集的最优特征的下标为（即决策树的根节点为）：{chooseBestFeatureToSplit(dataSet)}")
print(f"即：{labels[chooseBestFeatureToSplit(dataSet)]}")

当前数据集的最优特征的下标为（即决策树的根节点为）：0
即：no surfacing


##### 递归构建决策树

In [7]:
import operator
# 函数说明:统计classList中出现此处最多的元素(类标签)
def majorityCnt(classList):
    """
    Parameters:
    classList - 类标签列表
    Returns:
    sortedClassCount[0][0] - 出现此处最多的元素(类标签)
    """
    classCount = {}
    for vote in classList:#统计classList中每个元素出现的次数
        if vote not in classCount.keys():classCount[vote] = 0
        classCount[vote] += 1
    sortedClassCount = sorted(classCount.items(), key = operator.itemgetter(1), reverse = True)#根据字典的值降序排序
    return sortedClassCount[0][0]#返回classList中出现次数最多的元素


# 函数说明:创建决策树
def createTree(dataSet, labels):
    """
    Parameters:
    dataSet - 训练数据集
    labels - 分类属性标签
    Returns:
    myTree - 决策树
    """
    classList = [example[-1] for example in dataSet] #取分类标签(是否放贷:yes or no)
    if classList.count(classList[0]) == len(classList): #如果类别完全相同则停止继续划分
        return classList[0]
    
    if len(dataSet[0]) == 1: #遍历完所有特征时返回出现次数最多的类标签
        return majorityCnt(classList)
    
    bestFeat = chooseBestFeatureToSplit(dataSet) #选择最优特征
    bestFeatLabel = labels[bestFeat] #最优特征的标签
    myTree = {bestFeatLabel:{}} #根据最优特征的标签生成树

    del(labels[bestFeat]) #删除已经使用特征标签
    #（因为dataSet的bestFeat特征在划分子集时被删掉，所以labels也要删掉对应的特征名称）

    featValues = [example[bestFeat] for example in dataSet]#得到训练集中所有最优特征的属性值
    uniqueVals = set(featValues) #去掉重复的属性值

    for value in uniqueVals: #遍历特征，创建决策树。
        myTree[bestFeatLabel][value] = createTree(splitDataSet(dataSet, bestFeat, value), labels)
    return myTree

temp = labels[:]
myTree = createTree(dataSet, temp)
print(f"\n构建决策树如下:\n{myTree}")




构建决策树如下:
{'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}


##### 使用决策树分类

In [8]:
# 函数说明:使用决策树分类
def classify(inputTree,labels, testVec):
    """
    Parameters:
    inputTree - 已经生成的决策树
    testVec - 测试数据
    Returns:
    classLabel - 分类结果
    """
    firstStr = next(iter(inputTree)) #获取决策树结点
    #print(f"firstStr={firstStr}")
    secondDict = inputTree[firstStr] #下一个字典
    #print(f"secondDict={secondDict}")
    featIndex = labels.index(firstStr)
    #print(f"featIndex={featIndex}")
    for key in secondDict.keys():
        if testVec[featIndex] == key:
            if type(secondDict[key]).__name__ == 'dict':
                classLabel = classify(secondDict[key], labels, testVec)
            else: classLabel = secondDict[key]
    return classLabel

testVec = [1,1]#测试数据
result = classify(myTree,labels,testVec)
print(f"\n对数据{testVec}的分类结果为：\n{result}")

testVec = [1,0]#测试数据
result = classify(myTree,labels,testVec)
print(f"\n对数据{testVec}的分类结果为：\n{result}")



对数据[1, 1]的分类结果为：
yes

对数据[1, 0]的分类结果为：
no


### Sklearn实现决策树

```
from sklearn.tree import DecisionTreeClassifier

classfier = DecisionTreeClassifier()

classfier.fit(X_train, y_train)

y_pred = clf.predict(X_test)
```

![image.png](6.png)